In [ ]:
import os

import pickle
import random
import numpy as np

import torch
from torch.autograd import Variable


In [ ]:
#from model import RN, CNN_MLP, RFS

import model

import importlib
importlib.reload(model)

In [ ]:
model.RFS
